In [ ]:

# === IMPORTS ===
import os
import sys
import pandas as pd
import numpy as np

# Add parent directory to path to import utils
parent_dir = os.path.dirname(os.path.dirname(os.path.abspath('')))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from model_utils import get_device, create_pipeline, TOKENIZER_NAMES
from utils import (
    calcular_resultados, apply_protocol, load_enem_dataset,
    PROTOCOL_LABELS, PROTOCOL_ORDER, generate_latex_table,
    plot_real_vs_predicted, plot_confusion_matrix
)
from config import (
    setup_colab_paths, get_save_dir, MODEL_TEMPLATES,
    TEST_YEARS, COMPETENCIES
)
from tqdm.notebook import tqdm

In [ ]:
# === SETUP PATHS ===
# Setup Google Colab paths (optional - will work without Colab too)
try:
    DRIVE_BASE_PATH = setup_colab_paths(mount_drive=True)
except:
    # If not in Colab, use local paths
    DRIVE_BASE_PATH = os.path.join(os.getcwd(), "results")
    os.makedirs(DRIVE_BASE_PATH, exist_ok=True)
    print(f"✓ Using local path: {DRIVE_BASE_PATH}")

SAVE_DIR = get_save_dir(DRIVE_BASE_PATH, "inferencia_modelos_jbsc")
print(f"✓ Diretório de predições: {SAVE_DIR}")

In [ ]:

# === CARREGAR DATASET ===
# Load dataset and get test split
_, df_test = load_enem_dataset(anos_teste=TEST_YEARS)
print(f"✓ Dataset de teste carregado: {len(df_test)} redações")
df_test.head()


In [ ]:

# === CONFIGURAÇÃO DOS MODELOS ===
device = get_device()
print(f"Using device: {device}")

# Model configurations
model_types = MODEL_TEMPLATES["jbsc_original"]
competencias = COMPETENCIES

# Load pipelines
pipelines = {}
for model_key, model_path_template in model_types.items():
    print(f"\n📦 Carregando modelos: {model_key}")
    pipelines[model_key] = {}
    
    tokenizer_name = TOKENIZER_NAMES.get(model_key)
    
    for c in competencias:
        comp_key = f"C{c}"
        try:
            model_name = model_path_template.format(c)
            print(f"  {comp_key} | Modelo: {model_name}")
            
            pipe = create_pipeline(
                model_name,
                tokenizer_name=tokenizer_name,
                device=device,
                max_len=512
            )
            
            pipelines[model_key][comp_key] = pipe
        except Exception as e:
            print(f"  ❌ Erro ao carregar modelo {model_name}: {e}")
            pipelines[model_key][comp_key] = None

print("\n✅ Carregamento concluído!")



In [ ]:
# Previsões com modelo m-bert
model_key = "bert-base"

# Verificar se já existe CSV com previsões
if 'SAVE_DIR' not in globals():
    SAVE_DIR = os.path.join(DRIVE_BASE_PATH, "inferencia_modelos_jbsc")
    os.makedirs(SAVE_DIR, exist_ok=True)

csv_path = os.path.join(SAVE_DIR, "predicoes_mbert_jbsc_conjunto_teste.csv")

if os.path.exists(csv_path):
    print(f"✅ Carregando previsões existentes de {csv_path}")
    df_base = pd.read_csv(csv_path)
    print(f"   Carregadas {len(df_base)} previsões")
else:
    print(f"🔎 Rodando previsões com {model_key}...")
    textos = dataset["texto"]
    resultados = {f"C{c}": [] for c in competencias}

    for texto in tqdm(textos, desc=f"Processando com {model_key}"):
        for c in competencias:
            comp_key = f"C{c}"
            pred = pipelines[model_key][comp_key](texto)[0]
            nota = int(pred["label"])
            resultados[comp_key].append(nota)

    # Criar nova versão do dataset com predições
    df_base = dataset.to_pandas().copy()
    for c in competencias:
        comp_key = f"C{c}"
        df_base[f"pred_{comp_key}"] = resultados[comp_key]

    df_base["nota_final_predita"] = df_base[[f"pred_C{c}" for c in competencias]].sum(axis=1)

    # Salvar no Google Drive
    df_base.to_csv(csv_path, index=False)
    print(f"✅ Previsões salvas em: {csv_path}")



In [ ]:
# Previsões com modelo bertugues
model_key = "bertugues"

# Verificar se já existe CSV com previsões
if 'SAVE_DIR' not in globals():
    SAVE_DIR = os.path.join(DRIVE_BASE_PATH, "inferencia_modelos_jbsc")
    os.makedirs(SAVE_DIR, exist_ok=True)

csv_path = os.path.join(SAVE_DIR, "predicoes_bertugues_jbsc_conjunto_teste.csv")

if os.path.exists(csv_path):
    print(f"✅ Carregando previsões existentes de {csv_path}")
    df_bertugues = pd.read_csv(csv_path)
    print(f"   Carregadas {len(df_bertugues)} previsões")
else:
    print(f"🔎 Rodando previsões com {model_key}...")
    textos = dataset["texto"]
    resultados = {f"C{c}": [] for c in competencias}

    for texto in tqdm(textos, desc=f"Processando com {model_key}"):
        for c in competencias:
            comp_key = f"C{c}"
            pred = pipelines[model_key][comp_key](texto)[0]
            nota = int(pred["label"])
            resultados[comp_key].append(nota)

    # Criar nova versão do dataset com predições
    df_bertugues = dataset.to_pandas().copy()
    for c in competencias:
        comp_key = f"C{c}"
        df_bertugues[f"pred_{comp_key}"] = resultados[comp_key]

    df_bertugues["nota_final_predita"] = df_bertugues[[f"pred_C{c}" for c in competencias]].sum(axis=1)

    # Salvar no Google Drive
    df_bertugues.to_csv(csv_path, index=False)
    print(f"✅ Previsões salvas em: {csv_path}")


In [ ]:
# Previsões com modelo bertimbau
model_key = "bertimbau"

# Verificar se já existe CSV com previsões
if 'SAVE_DIR' not in globals():
    SAVE_DIR = os.path.join(DRIVE_BASE_PATH, "inferencia_modelos_jbsc")
    os.makedirs(SAVE_DIR, exist_ok=True)

csv_path = os.path.join(SAVE_DIR, "predicoes_bertimbau_jbsc_conjunto_teste.csv")

if os.path.exists(csv_path):
    print(f"✅ Carregando previsões existentes de {csv_path}")
    df_bertimbau = pd.read_csv(csv_path)
    print(f"   Carregadas {len(df_bertimbau)} previsões")
else:
    print(f"🔎 Rodando previsões com {model_key}...")
    textos = dataset["texto"]
    resultados = {f"C{c}": [] for c in competencias}

    for texto in tqdm(textos, desc=f"Processando com {model_key}"):
        for c in competencias:
            comp_key = f"C{c}"
            pred = pipelines[model_key][comp_key](texto)[0]
            nota = int(pred["label"])
            resultados[comp_key].append(nota)

    # Criar nova versão do dataset com predições
    df_bertimbau = dataset.to_pandas().copy()
    for c in competencias:
        comp_key = f"C{c}"
        df_bertimbau[f"pred_{comp_key}"] = resultados[comp_key]

    df_bertimbau["nota_final_predita"] = df_bertimbau[[f"pred_C{c}" for c in competencias]].sum(axis=1)

    # Salvar no Google Drive
    df_bertimbau.to_csv(csv_path, index=False)
    print(f"✅ Previsões salvas em: {csv_path}")


In [ ]:
import ast
import pandas as pd

competencias = [1, 2, 3, 4, 5]

def ajustar_para_correcao_dupla(y_true, y_pred):
    y_true_adj = []
    y_pred_adj = []

    for r, p in zip(y_true, y_pred):
        if pd.isna(r) or pd.isna(p):
            continue

        r = int(r)
        p = int(p)

        if r % 40 == 0:
            y_true_adj.extend([r, r])
            y_pred_adj.extend([p, p])
        else:
            baixo = (r // 40) * 40
            cima = baixo + 40
            y_true_adj.extend([baixo, cima])
            y_pred_adj.extend([p, p])

    return y_true_adj, y_pred_adj

avaliacoes_por_modelo = {}

for model_key in ["bert-base", "bertugues", "bertimbau"]:
    print(f"\n📊 Avaliando modelo: {model_key}")

    # Tentar usar variável local se disponível
    df_resultados = None


    try:
        if model_key == "bert-base" and 'df_base' in globals():
            df_resultados = df_base.copy()
        elif model_key == "bertugues" and 'df_bertugues' in globals():
            df_resultados = df_bertugues.copy()
        elif model_key == "bertimbau" and 'df_bertimbau' in globals():
            df_resultados = df_bertimbau.copy()
    except NameError:
        pass

    # Ler dados do CSV se variável local não estiver disponível
    if df_resultados is None:
        try:
            df_resultados = pd.read_csv(f"/content/drive/MyDrive/Enem Dataset/predicoes_{model_key}.csv")
            print(f"  ℹ️ Usando dados do CSV (variável local não encontrada)")
        except (FileNotFoundError, pd.errors.EmptyDataError) as e:
            print(f"⚠️ Não foi possível encontrar dados para {model_key}. Pulando.")
            continue

    df_resultados["notas"] = df_resultados["notas"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

    avaliacoes = {}

    for c in competencias:
        comp_key = f"C{c}"
        y_real = df_resultados["notas"].apply(lambda x: x[c-1])
        y_pred = df_resultados[f"pred_{comp_key}"]

        y_real, y_pred = ajustar_para_correcao_dupla(y_real, y_pred)

        if not y_real:
            print(f"⚠️ Nenhum dado válido para {comp_key}")
            continue

        resultados = calcular_resultados(y_real, y_pred)
        avaliacoes[comp_key] = resultados

    avaliacoes_por_modelo[model_key] = avaliacoes

    for comp_key, resultado in avaliacoes.items():
        print(f"\n🔎 Avaliação - {comp_key}")
        print(f"  Acurácia       (ACC): {resultado['ACC']*100:.2f}%")
        print(f"  RMSE              : {resultado['RMSE']:.2f}")
        print(f"  QWK               : {resultado['QWK']:.3f}")
        print(f"  Divergência (DIV) : {resultado['DIV']:.2f}%")
        print(f"  F1 Macro          : {resultado['F1-Macro']:.3f}")
        print(f"  F1 Weighted       : {resultado['F1-Weighted']:.3f}")
        print(f"  Agregado          : {resultado['Agregado']:.2f}")




In [ ]:
import ast
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

competencias = [1, 2, 3, 4, 5]

model_types = {
    "bert-base": "kamel-usp/jbcs2025_bert-base-multilingual-cased-encoder_classification-C{}-essay_only",
    "bertugues": "kamel-usp/jbcs2025_BERTugues-base-portuguese-cased-encoder_classification-C{}-essay_only",
    "bertimbau": "kamel-usp/jbcs2025_bertimbau_base-C{}"
}


def ajustar_para_correcao_dupla(y_true, y_pred):
    """
    Cenário 'dup_bounds':
      - Se a verdade já é múltipla de 40 -> duplica (r,r) e (p,p).
      - Caso contrário -> cria (baixo, cima) para a verdade e duplica p.
    """
    y_true_adj, y_pred_adj = [], []
    for r, p in zip(y_true, y_pred):
        if pd.isna(r) or pd.isna(p):
            continue
        r = int(r); p = int(p)
        if r % 40 == 0:
            y_true_adj.extend([r, r])
            y_pred_adj.extend([p, p])
        else:
            baixo = (r // 40) * 40
            cima  = baixo + 40
            y_true_adj.extend([baixo, cima])
            y_pred_adj.extend([p, p])
    return y_true_adj, y_pred_adj

def arredonda_verdade(y_true, modo):
    """
    Arredonda as notas verdade para múltiplos de 40.
    modo: 'floor' | 'ceil' | 'none'
    """
    y_true = pd.Series(y_true).dropna().astype(int)
    if modo == 'floor':
        return (np.floor(y_true / 40) * 40).astype(int).tolist()
    elif modo == 'ceil':
        return (np.ceil(y_true / 40) * 40).astype(int).tolist()
    elif modo == 'none':
        return y_true.tolist()
    else:
        raise ValueError("modo inválido")

def filtra_verdades_multiplas_40(y_true, y_pred):
    mask = (pd.Series(y_true).astype(int) % 40 == 0)
    y_true_f = pd.Series(y_true)[mask].astype(int).tolist()
    y_pred_f = pd.Series(y_pred)[mask].astype(int).tolist()
    return y_true_f, y_pred_f


avaliacoes_por_modelo = {}
resumo_qwk = []


protocol_labels = {
    "no_change": "Sem ajuste de escala",
    "dup_bounds": "Correção dupla (baixo/cima)",
    "truth_floor40": "Arred. verdade p/ baixo (40)",
    "truth_ceil40": "Arred. verdade p/ cima (40)",
    "only_true_mult40": "Apenas verdade múltipla de 40",
}
protocol_order = ["no_change", "dup_bounds", "truth_floor40", "truth_ceil40", "only_true_mult40"]

for model_key in ["bert-base", "bertugues", "bertimbau"]:
    print(f"\n📊 Avaliando modelo: {model_key}")


    df_resultados = None

    try:
        if model_key == "bert-base" and 'df_base' in globals():
            df_resultados = df_base.copy()
        elif model_key == "bertugues" and 'df_bertugues' in globals():
            df_resultados = df_bertugues.copy()
        elif model_key == "bertimbau" and 'df_bertimbau' in globals():
            df_resultados = df_bertimbau.copy()
    except NameError:
        pass


    if df_resultados is None:
        try:
            df_resultados = pd.read_csv(f"/content/drive/MyDrive/Enem Dataset/predicoes_{model_key}.csv")
            print(f"  ℹ️ Usando dados do CSV (variável local não encontrada)")
        except (FileNotFoundError, pd.errors.EmptyDataError) as e:
            print(f"⚠️ Não foi possível encontrar dados para {model_key}. Pulando.")
            continue


    if "notas" in df_resultados.columns:
        df_resultados["notas"] = df_resultados["notas"].apply(
            lambda x: ast.literal_eval(x) if isinstance(x, str) else x
        )


    esquemas = {
        "no_change": "Sem ajustar escalas",
        "dup_bounds": "Correção dupla (baixo/cima) e previsto duplicado",
        "truth_floor40": "Arredondar verdade para baixo (múltiplos de 40)",
        "truth_ceil40": "Arredondar verdade para cima (múltiplos de 40)",
        "only_true_mult40": "Apenas casos com verdade múltipla de 40",
    }

    avaliacoes_por_modelo[model_key] = {}

    for esquema_key, esquema_desc in esquemas.items():
        print(f"\n=== Esquema: {esquema_desc} ===")
        avaliacoes = {}
        qwk_vals = []

        for c in competencias:
            comp_key = f"C{c}"

            if "notas" in df_resultados.columns:
                y_real = df_resultados["notas"].apply(lambda x: x[c-1] if isinstance(x, (list, tuple)) else np.nan)
            else:

                y_real = df_resultados[comp_key] if comp_key in df_resultados.columns else pd.Series(dtype=float)


            pred_col = f"pred_{comp_key}"
            if pred_col not in df_resultados.columns:
                print(f"Coluna de previsão ausente para {comp_key} em {model_key}. Pulando.")
                continue
            y_pred = df_resultados[pred_col]


            if esquema_key == "no_change":

                pares = pd.DataFrame({"r": y_real, "p": y_pred}).dropna()
                y_r = pares["r"].astype(int).tolist()
                y_p = pares["p"].astype(int).tolist()

            elif esquema_key == "dup_bounds":
                y_r, y_p = ajustar_para_correcao_dupla(y_real, y_pred)

            elif esquema_key == "truth_floor40":
                pares = pd.DataFrame({"r": y_real, "p": y_pred}).dropna()
                y_r = arredonda_verdade(pares["r"], "floor")
                y_p = pares["p"].astype(int).tolist()

            elif esquema_key == "truth_ceil40":
                pares = pd.DataFrame({"r": y_real, "p": y_pred}).dropna()
                y_r = arredonda_verdade(pares["r"], "ceil")
                y_p = pares["p"].astype(int).tolist()

            elif esquema_key == "only_true_mult40":
                pares = pd.DataFrame({"r": y_real, "p": y_pred}).dropna()
                y_r, y_p = filtra_verdades_multiplas_40(pares["r"], pares["p"])

            else:
                raise ValueError("Esquema desconhecido.")

            if not y_r:
                print(f"Nenhum dado válido para {comp_key} ({esquema_desc})")
                continue


            qwk_step = 20 if esquema_key == "no_change" else 40
            resultado = calcular_resultados(y_r, y_p, qwk_step=qwk_step)
            avaliacoes[comp_key] = resultado
            qwk_vals.append(resultado["QWK"])


            print(f"\n --> Avaliação - {comp_key}")
            print(f"  Acurácia       (ACC): {resultado['ACC']*100:.2f}%")
            print(f"  RMSE              : {resultado['RMSE']:.2f}")
            print(f"  QWK               : {resultado['QWK']:.3f}")
            print(f"  Divergência (DIV) : {resultado['DIV']*100:.2f}%")
            print(f"  F1 Macro          : {resultado['F1-Macro']:.3f}")
            print(f"  F1 Weighted       : {resultado['F1-Weighted']:.3f}")
            print(f"  Agregado          : {resultado['Agregado']:.2f}")

        avaliacoes_por_modelo[model_key][esquema_key] = avaliacoes

        # guarda QWK médio para ranking
        if qwk_vals:
            resumo_qwk.append({
                "Modelo": model_key,
                "Esquema": esquema_desc,
                "QWK_médio": float(np.mean(qwk_vals)),
                "Qtd_comp_avaliadas": len(qwk_vals)
            })

# ranking por QWK
if resumo_qwk:
    rank = pd.DataFrame(resumo_qwk).sort_values(by=["QWK_médio"], ascending=False)
    print("\n Ranking por QWK médio (desempate por ordem de aparecimento):")
    print(rank.to_string(index=False))
else:
    print("\n Não foi possível compor o ranking (sem QWKs calculados).")


In [ ]:
# Calcular média dos QWK de cada modelo nos 5 protocolos
print("\n📊 Média dos QWK por Modelo (considerando todos os 5 protocolos):")
print("=" * 70)

medias_por_modelo = {}

for model_key in avaliacoes_por_modelo.keys():
    print(f"\n🔹 Modelo: {model_key.upper()}")
    print("-" * 70)

    # Calcular média por competência (nos 5 protocolos)
    medias_por_competencia = {}

    for c in competencias:
        comp_key = f"C{c}"
        qwk_comp = []

        # Coletar QWK desta competência em todos os protocolos
        for esquema_key in protocol_order:
            if esquema_key in avaliacoes_por_modelo[model_key]:
                avaliacoes = avaliacoes_por_modelo[model_key][esquema_key]
                if comp_key in avaliacoes and 'QWK' in avaliacoes[comp_key]:
                    qwk_comp.append(avaliacoes[comp_key]['QWK'])

        if qwk_comp:
            media_comp = np.mean(qwk_comp)
            medias_por_competencia[comp_key] = media_comp
            print(f"  {comp_key}: {media_comp:.4f} (média nos 5 protocolos)")

    # Calcular média geral (média das médias por competência)
    if medias_por_competencia:
        media_geral = np.mean(list(medias_por_competencia.values()))
        medias_por_modelo[model_key] = media_geral
        print(f"\n  📊 Média Geral: {media_geral:.4f} (média das médias por competência)")

print("\n" + "=" * 70)

# Criar DataFrame para visualização
if medias_por_modelo:
    df_medias = pd.DataFrame([
        {"Modelo": model, "Média_QWK": qwk}
        for model, qwk in medias_por_modelo.items()
    ]).sort_values(by="Média_QWK", ascending=False)

    print("\n📋 Tabela Resumo (Média Geral por Modelo):")
    print(df_medias.to_string(index=False))

In [ ]:
metrics = [
    ("QWK", "QWK"),
    ("F1-Macro", "F1 Macro"),
    ("F1-Weighted", "F1 Weighted"),
]

for model_key, protocolos in avaliacoes_por_modelo.items():
    for met_key, met_title in metrics:
        # monta DataFrame [linhas=protocolos legíveis, colunas=C1..C5]
        df_tab = pd.DataFrame(index=[protocol_labels[k] for k in protocol_order],
                              columns=[f"C{i}" for i in competencias], dtype=float)
        for sk in protocol_order:
            if sk not in protocolos:
                continue
            compdict = protocolos[sk]
            for c in competencias:
                ck = f"C{c}"
                if ck in compdict and met_key in compdict[ck]:
                    df_tab.loc[protocol_labels[sk], ck] = compdict[ck][met_key]

        df_print = df_tab.round(3)
        caption = f"{met_title} por competência para o modelo {model_key} nos diferentes protocolos de avaliação"
        label = f"tab:{met_key.replace('-','').replace(' ','').lower()}_{model_key.replace('-','_')}"
        print(f"\n=== Tabela LaTeX — {met_title} — {model_key} ===\n")
        print(df_print.to_latex(index=True, caption=caption, label=label, na_rep="--", float_format="%.3f"))

In [ ]:
import matplotlib.pyplot as plt
import ast
from sklearn.metrics import r2_score, cohen_kappa_score
from sklearn.linear_model import LinearRegression
import numpy as np


modelos_info = [
    ("df_base", "bert-base", "predicoes_mbert_jbsc_conjunto_teste.csv"),
    ("df_bertugues", "bertugues", "predicoes_bertugues_jbsc_conjunto_teste.csv"),
    ("df_bertimbau", "bertimbau", "predicoes_bertimbau_jbsc_conjunto_teste.csv")
]

# Labels para QWK (0 a 200, de 40 em 40) - correspondente às predições
ALL_LABELS = list(range(0, 201, 40))
competencias = [1, 2, 3, 4, 5]


modelos_processados = []

for df_var_name, modelo_nome, csv_filename in modelos_info:
    df_plot = None


    if df_var_name in globals():
        try:
            df_plot = globals()[df_var_name].copy()
            print(f"\n✅ Usando dados de {modelo_nome} da variável {df_var_name}")
        except:
            pass


    if df_plot is None:
        try:
            csv_path = os.path.join(SAVE_DIR, csv_filename)
            df_plot = pd.read_csv(csv_path)
            print(f"✅ Carregando dados de {modelo_nome} do CSV: {csv_path}")
        except Exception as e:
            print(f"⚠️ Não foi possível carregar dados para {modelo_nome}: {e}")
            continue

    if df_plot is not None:

        if "notas" in df_plot.columns:
            df_plot["notas"] = df_plot["notas"].apply(
                lambda x: ast.literal_eval(x) if isinstance(x, str) else x
            )

        r2_scores = {}
        qwk_scores = {}


        fig, axes = plt.subplots(1, 5, figsize=(30, 6))
        fig.suptitle(f'Real vs Predicted Scores by Competency - Model {modelo_nome}',
                     fontsize=18, fontweight='bold', y=1.02)


        dados_graficos = {}

        for c in competencias:
            comp_key = f"C{c}"


            if "notas" in df_plot.columns:
                y_real = df_plot["notas"].apply(lambda x: x[c-1] if isinstance(x, (list, tuple)) else np.nan)
            else:
                y_real = df_plot[comp_key] if comp_key in df_plot.columns else pd.Series(dtype=float)

            y_pred = df_plot[f"pred_{comp_key}"]

            pares = pd.DataFrame({"r": y_real, "p": y_pred}).dropna()
            y_real_clean = pares["r"].astype(int).values
            y_pred_clean = pares["p"].astype(int).values

            if len(y_real_clean) == 0:
                print(f"⚠️ Nenhum dado válido para {comp_key}")
                dados_graficos[comp_key] = None
                continue

            # Calcular QWK usando a mesma função das tabelas (protocolo "sem ajuste")
            resultado = calcular_resultados(y_real_clean.tolist(), y_pred_clean.tolist(), qwk_step=20)
            qwk = resultado['QWK']
            qwk_scores[comp_key] = qwk

            # Calcular regressão linear: Y = nota real, X = nota prevista
            X = y_pred_clean.reshape(-1, 1)
            y = y_real_clean
            reg = LinearRegression()
            reg.fit(X, y)
            slope = reg.coef_[0]
            intercept = reg.intercept_

            # Calcular R² usando score da regressão
            r2 = reg.score(X, y)
            r2_scores[comp_key] = r2


            x_line = np.array([0, 200])
            y_line = slope * x_line + intercept

            dados_graficos[comp_key] = {
                'y_real': y_real_clean,
                'y_pred': y_pred_clean,
                'r2': r2,
                'qwk': qwk,
                'slope': slope,
                'intercept': intercept,
                'x_line': x_line,
                'y_line': y_line
            }


        for idx, c in enumerate(competencias):
            comp_key = f"C{c}"
            ax = axes[idx]

            if dados_graficos[comp_key] is None:
                ax.text(0.5, 0.5, 'Sem dados', ha='center', va='center', transform=ax.transAxes)
                ax.set_title(f'{comp_key}', fontsize=14, fontweight='bold')
                continue

            dados = dados_graficos[comp_key]

            # Adicionar jitter aos pontos para melhor visualização
            np.random.seed(42 + idx)
            jitter_x = np.random.normal(0, 1.5, size=len(dados['y_pred']))
            jitter_y = np.random.normal(0, 1.5, size=len(dados['y_real']))

            y_pred_jittered = dados['y_pred'] + jitter_x  # X = nota prevista
            y_real_jittered = dados['y_real'] + jitter_y  # Y = nota real


            ax.scatter(y_pred_jittered, y_real_jittered, alpha=0.5, s=30, edgecolors='black', linewidths=0.3)

            # Adicionar linha de referência (y = x)
            ax.plot([0, 200], [0, 200], 'r--', linewidth=2, label='y = x', alpha=0.8)

            # Adicionar linha de regressão linear (X = nota prevista, Y = nota real)
            ax.plot(dados['x_line'], dados['y_line'], 'b-', linewidth=2,
                    label=f'Regressão: y = {dados["slope"]:.3f}x + {dados["intercept"]:.2f}', alpha=0.8)


            ax.set_xlabel('Predicted Score', fontsize=12, fontweight='bold')
            if idx == 0:
                ax.set_ylabel('Real Score', fontsize=12, fontweight='bold')
            ax.set_title(f'{comp_key}\nR² = {dados["r2"]:.4f} | QWK = {dados["qwk"]:.4f}',
                         fontsize=13, fontweight='bold')


            ax.set_xlim(-10, 210)
            ax.set_xticks(range(0, 201, 40))
            ax.set_xticklabels(range(0, 201, 40), rotation=45, ha='right', fontsize=9)


            ax.set_ylim(-10, 210)
            ax.set_yticks(range(0, 201, 20))
            ax.set_yticklabels(range(0, 201, 20), fontsize=9)


            ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.8)


            ax.legend(loc='upper left', fontsize=8, framealpha=0.9)


            ax.set_aspect('equal', adjustable='box')

        plt.tight_layout()

        # Salvar gráfico completo
        plot_path = os.path.join(SAVE_DIR, f"graficos_real_vs_pred_{modelo_nome.replace('-', '_')}_completo.png")
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        print(f"✅ Gráfico completo salvo em: {plot_path}")


        plt.show()


        print(f"\n📊 Resumo das Métricas por Competência - {modelo_nome.upper()}:")
        print("=" * 60)
        print(f"{'Competência':<12} {'R²':<12} {'QWK':<12}")
        print("-" * 60)
        for comp_key in competencias:
            comp_key_str = f"C{comp_key}"
            if comp_key_str in r2_scores and comp_key_str in qwk_scores:
                print(f"{comp_key_str:<12} {r2_scores[comp_key_str]:<12.4f} {qwk_scores[comp_key_str]:<12.4f}")
        print("-" * 60)
        if r2_scores and qwk_scores:
            print(f"{'MÉDIO':<12} {np.mean(list(r2_scores.values())):<12.4f} {np.mean(list(qwk_scores.values())):<12.4f}")
        print("=" * 60)

        modelos_processados.append(modelo_nome)

# Resumo final
if modelos_processados:
    print(f"\n Gráficos gerados para {len(modelos_processados)} modelo(s): {', '.join(modelos_processados)}")
else:
    print("\n Nenhum modelo foi processado")


In [ ]:
# === CONFUSION MATRICES ===
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_recall_fscore_support
import seaborn as sns

# List of models to process
modelos_info = [
    ("df_base", "bert-base", "predicoes_mbert_jbsc_conjunto_teste.csv"),
    ("df_bertugues", "bertugues", "predicoes_bertugues_jbsc_conjunto_teste.csv"),
    ("df_bertimbau", "bertimbau", "predicoes_bertimbau_jbsc_conjunto_teste.csv")
]

competencias = [1, 2, 3, 4, 5]

# Possible labels (grades from 0 to 200, in steps of 20)
labels_possiveis = list(range(0, 201, 20))  # [0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200]

def arredondar_para_multiplo_20(nota):
    """Round grade to the nearest multiple of 20"""
    return int(round(nota / 20) * 20)

# Process each model
for df_var_name, modelo_nome, csv_filename in modelos_info:
    print(f"\n{'='*80}")
    print(f"📊 CONFUSION MATRICES - Model: {modelo_nome.upper()}")
    print(f"{'='*80}")

    df_plot = None

    # Try to use local variable first
    if df_var_name in globals():
        try:
            df_plot = globals()[df_var_name].copy()
            print(f"✅ Using data from {modelo_nome} from variable {df_var_name}")
        except:
            pass

    # Fallback: try to read from CSV
    if df_plot is None:
        try:
            csv_path = os.path.join(SAVE_DIR, csv_filename)
            df_plot = pd.read_csv(csv_path)
            print(f"✅ Loading data from {modelo_nome} from CSV: {csv_path}")
        except Exception as e:
            print(f"⚠️ Could not load data for {modelo_nome}: {e}")
            continue

    if df_plot is None:
        continue

    # Process 'notas' column if it exists
    if "notas" in df_plot.columns:
        df_plot["notas"] = df_plot["notas"].apply(
            lambda x: ast.literal_eval(x) if isinstance(x, str) else x
        )

    metricas_por_competencia = {}

    # Plot each confusion matrix separately
    for idx, c in enumerate(competencias):
        comp_key = f"C{c}"

        # Create a separate figure for each competence
        fig, ax = plt.subplots(1, 1, figsize=(10, 8))
        fig.suptitle(f'Confusion Matrix - {comp_key} - Model {modelo_nome} (Zero Shot JBSC)',
                     fontsize=16, fontweight='bold', y=0.98)

        # Extract real and predicted grades
        if "notas" in df_plot.columns:
            y_real = df_plot["notas"].apply(lambda x: x[c-1] if isinstance(x, (list, tuple)) else np.nan)
        else:
            y_real = df_plot[comp_key] if comp_key in df_plot.columns else pd.Series(dtype=float)

        y_pred = df_plot[f"pred_{comp_key}"]

        # Remove NaN and convert to int
        pares = pd.DataFrame({"r": y_real, "p": y_pred}).dropna()
        y_real_clean = pares["r"].astype(int).values
        y_pred_clean = pares["p"].astype(int).values

        if len(y_real_clean) == 0:
            ax.text(0.5, 0.5, 'No data', ha='center', va='center', transform=ax.transAxes, fontsize=14)
            ax.set_title(f'{comp_key}', fontsize=14, fontweight='bold')
            continue

        # Round both grades to multiples of 20
        y_real_clean = np.array([arredondar_para_multiplo_20(n) for n in y_real_clean])
        y_pred_clean = np.array([arredondar_para_multiplo_20(n) for n in y_pred_clean])

        # Ensure grades are in valid range [0, 200]
        y_real_clean = np.clip(y_real_clean, 0, 200)
        y_pred_clean = np.clip(y_pred_clean, 0, 200)

        # Calculate confusion matrix using ALL labels in steps of 20 (0 to 200)
        # This ensures all matrices have the same structure and are comparable
        cm = confusion_matrix(y_real_clean, y_pred_clean, labels=labels_possiveis)

        # Calculate QWK using calcular_resultados with qwk_step=20
        resultado = calcular_resultados(y_real_clean.tolist(), y_pred_clean.tolist(), qwk_step=20)
        qwk = resultado['QWK']

        # Calculate important metrics
        accuracy = accuracy_score(y_real_clean, y_pred_clean)

        # For precision, recall and f1, use average='weighted' to handle imbalanced classes
        precision, recall, f1, support = precision_recall_fscore_support(
            y_real_clean, y_pred_clean, average='weighted', zero_division=0
        )

        # Calculate macro average as well
        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
            y_real_clean, y_pred_clean, average='macro', zero_division=0
        )

        metricas_por_competencia[comp_key] = {
            'accuracy': accuracy,
            'qwk': qwk,
            'precision_weighted': precision,
            'recall_weighted': recall,
            'f1_weighted': f1,
            'precision_macro': precision_macro,
            'recall_macro': recall_macro,
            'f1_macro': f1_macro,
            'total_samples': len(y_real_clean)
        }

        # Create heatmap of confusion matrix
        # Use size proportional to number of samples
        # Use all labels in steps of 20 for complete matrix
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=labels_possiveis, yticklabels=labels_possiveis,
                   ax=ax, cbar_kws={'label': 'Count'},
                   linewidths=0.5, linecolor='gray')

        ax.set_xlabel('Predicted Grade', fontsize=12, fontweight='bold')
        ax.set_ylabel('True Grade', fontsize=12, fontweight='bold')
        ax.set_title(f'{comp_key}', fontsize=14, fontweight='bold')

        # Add metrics as text below the plot
        metricas_texto = f'Acc: {accuracy:.3f} | QWK: {qwk:.3f} | F1-W: {f1:.3f} | F1-M: {f1_macro:.3f}'
        ax.text(0.5, -0.12, metricas_texto, transform=ax.transAxes,
               ha='center', fontsize=10, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

        plt.tight_layout()

        # Save individual plot
        plot_path = os.path.join(SAVE_DIR, f"confusion_matrix_{modelo_nome}_{comp_key}_zero_shot_jbsc.png")
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        print(f"✅ Confusion matrix for {comp_key} saved to: {plot_path}")

        plt.show()

    # Print summary of metrics
    print(f"\n📊 Summary of Metrics by Competence - {modelo_nome.upper()}:")
    print("=" * 120)
    print(f"{'Competence':<12} {'Accuracy':<10} {'QWK':<10} {'Precision-W':<12} {'Recall-W':<12} {'F1-Weighted':<12} {'F1-Macro':<12} {'Samples':<10}")
    print("-" * 120)
    for comp_key in competencias:
        comp_key_str = f"C{comp_key}"
        if comp_key_str in metricas_por_competencia:
            m = metricas_por_competencia[comp_key_str]
            print(f"{comp_key_str:<12} {m['accuracy']:<10.4f} {m['qwk']:<10.4f} {m['precision_weighted']:<12.4f} {m['recall_weighted']:<12.4f} "
                  f"{m['f1_weighted']:<12.4f} {m['f1_macro']:<12.4f} {m['total_samples']:<10}")
    print("-" * 120)
    if metricas_por_competencia:
        avg_acc = np.mean([m['accuracy'] for m in metricas_por_competencia.values()])
        avg_qwk = np.mean([m['qwk'] for m in metricas_por_competencia.values()])
        avg_f1_w = np.mean([m['f1_weighted'] for m in metricas_por_competencia.values()])
        avg_f1_m = np.mean([m['f1_macro'] for m in metricas_por_competencia.values()])
        total_samples = sum([m['total_samples'] for m in metricas_por_competencia.values()])
        print(f"{'AVERAGE':<12} {avg_acc:<10.4f} {avg_qwk:<10.4f} {'-':<12} {'-':<12} {avg_f1_w:<12.4f} {avg_f1_m:<12.4f} {total_samples:<10}")
    print("=" * 120)
